In [ ]:
def make_checkm_output_df(filedir):
    # run the read_checkm_tsv and add the outputfile.txt
    meta = read_checkm_tsv(filedir)
    strhet = pd.read_table(filedir+'outputfile.txt')
    strhet = strhet.set_index('Bin Id')
    
    meta = meta.join(strhet['Strain heterogeneity'])
    meta = meta.sort_values('Completeness',ascending=False)
    return(meta)

In [ ]:
# plot the contigs of each genome bin seperately and color according to their being selected
# for final bin or not (multiple colors for bin splits)
# here add the checkM results for a particular bin 
man_genome_contigs['name'] = man_genome_contigs['Sample']+'_genome_'+man_genome_contigs['OriginalGenomeID']

# get the relevant metadata files 
metaTake2 = make_checkm_output_df('Checkm/Manual_genomes_obs_take2/')
metaTake2_oidx = get_old_bin_names(metaTake2,26,29)
reassem = make_checkm_output_df('Checkm/reassembly_3kb/')
reassem_oldidx = get_old_bin_names(reassem,26,29)
orig_bins = make_checkm_output_df('Checkm/obsidian_all_checkm/')
orig_bins = orig_bins.loc[alt5.index,:]
transl = pd.read_table('fasta/genome_bins_obsidian/info.table')
transl['brian_name'] = 'Obsidian'+transl['sample'].str[-1]+'_genome_'+transl['NewName'].str[-3:]
Qfilter = pd.DataFrame(index=transl['brian_name'],columns=['old_name','original','reassembled','c1','c2','c3','c4'])
Qfilter['old_name'] = [transl[transl['brian_name']==name]['Unnamed: 0'].values[0] for name in Qfilter.index]#[s for s in alt5.index]

for name in man_genome_contigs['name'].unique():
    tempdf = man_genome_contigs[man_genome_contigs['name']==name]
    one = 0;two = 0;three = 0;four = 0
    f = plt.figure()
    f.set_figwidth(9)
    f.set_figheight(9)
    
    gs = gridspec.GridSpec(4,4)
    
    ax1 = f.add_subplot(gs[0:2,0:2])
    if len(tempdf[tempdf['ManualGenomeID'].str[-1]=='1'])>0:
        t1 = tempdf[tempdf['ManualGenomeID'].str[-1]=='1']
        t1.plot.scatter('GC','FPK',s=np.array(t1['ContigLength'])/3e2, alpha=0.5,color='blue',ax=ax1,fontsize=10)
        one = 1
    if len(tempdf[tempdf['ManualGenomeID'].str[-1]=='2'])>0:
        t2 = tempdf[tempdf['ManualGenomeID'].str[-1]=='2']
        t2.plot.scatter('GC','FPK',s=np.array(t2['ContigLength'])/3e2,alpha=0.5,color='red',ax=ax1,fontsize=10)
        two = 1
    if len(tempdf[tempdf['ManualGenomeID'].str[-1]=='3'])>0:
        t3 = tempdf[tempdf['ManualGenomeID'].str[-1]=='3']
        t3.plot.scatter('GC','FPK',s=np.array(t3['ContigLength'])/3e2,alpha=0.5,color='g',ax=ax1,fontsize=10)
        three = 1
    if len(tempdf[tempdf['ManualGenomeID'].str[-1]=='4'])>0:
        t4 = tempdf[tempdf['ManualGenomeID'].str[-1]=='4']
        t4.plot.scatter('GC','FPK',s=np.array(t4['ContigLength'])/3e2,alpha=0.5,color='m',ax=ax1,fontsize=10)
        four = 1
    if len(tempdf[tempdf['IsDropped']=='Yes'])>0:
        t0 = tempdf[tempdf['IsDropped']=='Yes']
        t0.plot.scatter('GC','FPK',s=np.array(t0['ContigLength'])/3e2,alpha=0.3,color='k',ax=ax1,fontsize=10)
    plt.xlabel('GC',fontsize=12)
    plt.ylim(1e0,1e6);plt.xlim(0.2,0.8)
    plt.yscale('log')
    plt.ylabel('Fragments per kbp',fontsize=12)
    plt.title(name,fontsize=13)
    
    if one == 1:
        ax2 = f.add_subplot(gs[0,2]) # this is bin #1 
        data = metaTake2[metaTake2.index.str[26:29] == name[-3:]]
        data = data[data.index.str[-1] == '1']
        data[['Completeness','Contamination','Strain heterogeneity']].plot.bar(ax=ax2,fontsize=10)
        plt.ylim(0,100)
        ax2.legend(bbox_to_anchor=(0.1, 1),loc=3, ncol=1, mode="expand", borderaxespad=0.,fontsize=10)
        plt.xticks([]);plt.xlabel('');
        ax2.text(0.7,30,
                 'lineage:   '+data['marker lineage'][0]
                 +'\nlength (Mb):       '+str(round(data['Genome size'][0]/1e6,2))
                 +'\n# contigs:           '+data['# contigs'].astype(str)[0]
                 +'\nN50 (kb):            '+str(round(data['N50 (contigs)'][0]/1e3,1)),
                 fontsize=12,color='b')
        if (data['Completeness'][0]>10) & (data['Contamination'][0]<5) & (data['Strain heterogeneity'][0]<5):
            Qfilter.loc[name,'c1'] = data['Completeness'][0]
    if two == 1:
        ax3 = f.add_subplot(gs[1,2]) # this is bin #2
        data = metaTake2[metaTake2.index.str[26:29] == name[-3:]]
        data = data[data.index.str[-1] == '2']
        data[['Completeness','Contamination','Strain heterogeneity']].plot.bar(ax=ax3,legend=False)
        plt.xticks([]);plt.ylim(0,100);plt.xlabel('');plt.yticks([])
        ax3.text(0.7,30,
                 'lineage:   '+data['marker lineage'][0]
                 +'\nlength (Mb):       '+str(round(data['Genome size'][0]/1e6,2))
                 +'\n# contigs:           '+data['# contigs'].astype(str)[0]
                 +'\nN50 (kb):            '+str(round(data['N50 (contigs)'][0]/1e3,1)),
                 fontsize=12,color='r')
        if (data['Completeness'][0]>10) & (data['Contamination'][0]<5) & (data['Strain heterogeneity'][0]<5):
            Qfilter.loc[name,'c2'] = data['Completeness'][0]
    if three == 1:
        ax4 = f.add_subplot(gs[2,2]) # this is bin #3
        data = metaTake2[metaTake2.index.str[26:29] == name[-3:]]
        data = data[data.index.str[-1] == '3']
        data[['Completeness','Contamination','Strain heterogeneity']].plot.bar(ax=ax4,legend=False,fontsize=10)
        plt.xticks([]);plt.ylim(0,100);plt.xlabel('');plt.yticks([])
        ax4.text(0.7,30,
                 'lineage:   '+data['marker lineage'][0]
                 +'\nlength (Mb):       '+str(round(data['Genome size'][0]/1e6,2))
                 +'\n# contigs:           '+data['# contigs'].astype(str)[0]
                 +'\nN50 (kb):            '+str(round(data['N50 (contigs)'][0]/1e3,1)),
                 fontsize=12,color='g')
        if (data['Completeness'][0]>10) & (data['Contamination'][0]<5) & (data['Strain heterogeneity'][0]<5):
            Qfilter.loc[name,'c3'] = data['Completeness'][0]
    if four == 1:
        ax5 = f.add_subplot(gs[3,2]) # this is bin #3
        data = metaTake2[metaTake2.index.str[26:29] == name[-3:]]
        data = data[data.index.str[-1] == '4']
        data[['Completeness','Contamination','Strain heterogeneity']].plot.bar(ax=ax5,legend=False)
        plt.xticks([]);plt.ylim(0,100);plt.xlabel('');plt.yticks([])
        ax5.text(0.7,30,
                 'lineage:   '+data['marker lineage'][0]
                 +'\nlength (Mb):       '+str(round(data['Genome size'][0]/1e6,2))
                 +'\n# contigs:           '+data['# contigs'].astype(str)[0]
                 +'\nN50 (kb):            '+str(round(data['N50 (contigs)'][0]/1e3,1)),
                 fontsize=12,color='m')
        if (data['Completeness'][0]>10) & (data['Contamination'][0]<5) & (data['Strain heterogeneity'][0]<5):
            Qfilter.loc[name,'c4'] = data['Completeness'][0]
    ax6 = f.add_subplot(gs[2,0])
    data = reassem[reassem.index.str[26:29]==name[-3:]]
    data[['Completeness','Contamination','Strain heterogeneity']].plot.bar(ax=ax6,legend=False,fontsize=10)
    plt.xticks([]);plt.ylim(0,100);plt.xlabel('')#;plt.yticks([])
    ax6.text(0.6,20,
                 'lineage:   '+data['marker lineage'][0]
                 +'\nlength (Mb):     '+str(round(data['Genome size'][0]/1e6,2))
                 +'\n# contigs:         '+data['# contigs'].astype(str)[0]
                 +'\nN50 (kb):          '+str(round(data['N50 (contigs)'][0]/1e3,1)),
                 fontsize=12)
    plt.ylabel('Reassembly\nnon-curated',fontsize=12)
    if (data['Completeness'][0]>10) & (data['Contamination'][0]<5) & (data['Strain heterogeneity'][0]<5):
        Qfilter.loc[name,'reassembled'] = data['Completeness'][0]
    ax7 = f.add_subplot(gs[3,0])
    data = pd.DataFrame(orig_bins.loc[reassem_oldidx[reassem_oldidx['new_idx'].str[26:29]==name[-3:]].index[0][0:-2]]).T
    data[['Completeness','Contamination','Strain heterogeneity']].plot.bar(ax=ax7,legend=False,fontsize=10)
    plt.xticks([]);plt.ylim(0,100);plt.xlabel('');#plt.yticks([])
    ax7.text(0.6,20,
                 'lineage:   '+data['marker lineage'][0]
                 +'\nlength (Mb):     '+str(round(data['Genome size'][0]/1e6,2))
                 +'\n# contigs:         '+data['# contigs'].astype(str)[0]
                 +'\nN50 (kb):          '+str(round(data['N50 (contigs)'][0]/1e3,1)),
                 fontsize=12)
    plt.title('Original bin: '+data.index[0],fontsize=12)
    if (data['Completeness'][0]>10) & (data['Contamination'][0]<5) & (data['Strain heterogeneity'][0]<5):
        Qfilter.loc[name,'original'] = data['Completeness'][0]

    gs.update(hspace=.2)#,wspace=0.02)
    plt.gcf().subplots_adjust(bottom=0.03)
    f.savefig('reassembly/Manual_Genomes_take2//plots_QS/'+name) #this is plots+ quality score
    f.clf()